<a href="https://colab.research.google.com/github/IamSavitha/Datawarehouse/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import os

os.environ["ALPHA_VANTAGE_API_KEY"] = "N9LNZ3Q3I1QALQOV"
os.environ["SNOWFLAKE_USER"] = "SAVITHAVIJAYARANGAN"
os.environ["SNOWFLAKE_PASSWORD"] = "Studywell12345$"
os.environ["SNOWFLAKE_ACCOUNT"] = "TRB46163"



In [23]:
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")
SNOWFLAKE_ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")


In [24]:
import requests
import pandas as pd

# Define stock symbol
STOCK_SYMBOL = "AAPL"  # Change to any stock you prefer

# Alpha Vantage API URL
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={STOCK_SYMBOL}&apikey={ALPHA_VANTAGE_API_KEY}&outputsize=compact"

# Fetch data
response = requests.get(url)
data = response.json()

# Extract time series data
time_series = data.get("Time Series (Daily)", {})

# Convert to DataFrame
df = pd.DataFrame.from_dict(time_series, orient="index")
df.reset_index(inplace=True)
df.columns = ["date", "open", "high", "low", "close", "volume"]

# Convert data types
df["date"] = pd.to_datetime(df["date"])
df[["open", "high", "low", "close", "volume"]] = df[["open", "high", "low", "close", "volume"]].astype(float)

# Keep only last 90 days
df = df.sort_values("date", ascending=False).head(90)

print(df.head())  # Display first few rows


        date    open    high       low   close      volume
0 2025-02-21  245.95  248.69  245.2200  245.55  53197431.0
1 2025-02-20  244.94  246.78  244.2900  245.83  32316907.0
2 2025-02-19  244.66  246.01  243.1604  244.87  32204215.0
3 2025-02-18  244.15  245.18  241.8400  244.47  48822491.0
4 2025-02-14  241.25  245.55  240.9900  244.60  40896227.0


In [25]:
!pip install snowflake-connector-python

import snowflake.connector

# Establish connection
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT
)

# Create a cursor
cur = conn.cursor()


In [26]:
import snowflake.connector
import os

# Corrected Snowflake Account Identifier
SNOWFLAKE_ACCOUNT = "trb46163.us-west-2"

# Securely retrieve credentials
SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT
)

cur = conn.cursor()
print("Connected to Snowflake!")


Connected to Snowflake!


In [27]:
# Step 1: Select your database (replace 'mydatabase' with your actual database name)
cur.execute("USE DATABASE mydatabase")

# Step 2: Create the schema if it doesn’t exist
cur.execute("CREATE SCHEMA IF NOT EXISTS raw")

# Step 3: Create the table inside the 'raw' schema
create_table_query = """
CREATE TABLE IF NOT EXISTS raw.stock_data (
    symbol STRING,
    date DATE,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume FLOAT,
    PRIMARY KEY (symbol, date)
)
"""
cur.execute(create_table_query)

print("Table 'raw.stock_data' created or already exists.")


Table 'raw.stock_data' created or already exists.


In [28]:
# Show tables in the 'raw' schema
cur.execute("SHOW TABLES IN raw")

In [29]:
tables = cur.fetchall()
for table in tables:
    print(table)


(datetime.datetime(2025, 2, 22, 11, 30, 10, 840000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), 'STOCK_DATA', 'MYDATABASE', 'RAW', 'TABLE', '', '', 90, 6144, 'ACCOUNTADMIN', '1', 'OFF', 'OFF', 'OFF', None, None, 'N', 'N', 'ROLE', 'N', None, 'N', 'N', 'N', 'N')


In [30]:
delete_query = "DELETE FROM raw.stock_data WHERE symbol = %s"
cur.execute(delete_query, (STOCK_SYMBOL,))



In [31]:
try:
    cur.execute("BEGIN")  # Start transaction

    insert_query = """
        INSERT INTO raw.stock_data (symbol, date, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """

    # Loop through DataFrame and insert data
    for _, row in df.iterrows():
        cur.execute(insert_query, (
            STOCK_SYMBOL,
            row["date"].strftime('%Y-%m-%d'),  # Convert date to string
            row["open"],
            row["high"],
            row["low"],
            row["close"],
            row["volume"]
        ))

    cur.execute("COMMIT")  # Commit transaction
    print("Data inserted successfully!")

except Exception as e:
    cur.execute("ROLLBACK")  # Rollback in case of an error
    print("Error occurred:", e)


Data inserted successfully!


In [32]:
cur.execute("SELECT COUNT(*) FROM raw.stock_data WHERE symbol = %s", (STOCK_SYMBOL,))
record_count = cur.fetchone()
print(f"Total records after re-running: {record_count[0]}")


Total records after re-running: 90
